In [1]:
import numpy as np
import scipy.stats as scs
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [38]:
from sklearn.preprocessing import LabelBinarizer

In [2]:
raw_df = pd.read_csv('/mnt/c/Users/kurtrm/Downloads/bigml_5acff779eba31d156f0000fa.csv')

In [3]:
raw_df['Failure'].value_counts()

No     8703
Yes      81
Name: Failure, dtype: int64

In [4]:
raw_df.columns

Index(['Date', 'Temperature', 'Humidity', 'Operator', 'Measure1', 'Measure2',
       'Measure3', 'Measure4', 'Measure5', 'Measure6', 'Measure7', 'Measure8',
       'Measure9', 'Measure10', 'Measure11', 'Measure12', 'Measure13',
       'Measure14', 'Measure15', 'Hours Since Previous Failure', 'Failure',
       '﻿Date.year', '﻿Date.month', '﻿Date.day-of-month', '﻿Date.day-of-week',
       '﻿Date.hour', '﻿Date.minute', '﻿Date.second'],
      dtype='object')

Fifteen columns have generic names. In the spirit of creativity, I renamed these columns to readings that we may actually see on equipment logs, though their values may be unrealistic to what we could expect of these readings. 

In [5]:
raw_df[[f'Measure{i}' for i in range(1, 16)]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 15 columns):
Measure1     8784 non-null int64
Measure2     8784 non-null int64
Measure3     8784 non-null int64
Measure4     8784 non-null int64
Measure5     8784 non-null int64
Measure6     8784 non-null int64
Measure7     8784 non-null int64
Measure8     8784 non-null int64
Measure9     8784 non-null int64
Measure10    8784 non-null int64
Measure11    8784 non-null int64
Measure12    8784 non-null int64
Measure13    8784 non-null int64
Measure14    8784 non-null int64
Measure15    8784 non-null int64
dtypes: int64(15)
memory usage: 1.0 MB


In [6]:
fake_categories = ['discharge_pressure', 'suction_pressure', 'differential_pressure', 'rotor_temperature', 'packing_leakage_rate',
                   'axial_bearing_temp', 'horizontal_bearing_temp', 'frequency', 'noise', 'voltage', 'current', 'speed',
                   'system_pressure', 'suction_fluid_temperature', 'discharge_fluid_temperature']

In [7]:
replacements = {key: value for key, value in zip([f'Measure{i}' for i in range(1, 16)], fake_categories)}
new_columns = [name if name not in replacements else replacements[name] for name in raw_df.columns]

In [8]:
new_columns

['Date',
 'Temperature',
 'Humidity',
 'Operator',
 'discharge_pressure',
 'suction_pressure',
 'differential_pressure',
 'rotor_temperature',
 'packing_leakage_rate',
 'axial_bearing_temp',
 'horizontal_bearing_temp',
 'frequency',
 'noise',
 'voltage',
 'current',
 'speed',
 'system_pressure',
 'suction_fluid_temperature',
 'discharge_fluid_temperature',
 'Hours Since Previous Failure',
 'Failure',
 '\ufeffDate.year',
 '\ufeffDate.month',
 '\ufeffDate.day-of-month',
 '\ufeffDate.day-of-week',
 '\ufeffDate.hour',
 '\ufeffDate.minute',
 '\ufeffDate.second']

There are some odd strings in the 'Date.' categories. I'll clean those up.

In [17]:
new_columns[-7:] = ['year', 'month', 'day-of-month', 'day-of-week', 'hour', 'minute', 'second']

In [19]:
raw_df.columns = new_columns

Next, change the 'Date' column datatype to datetime objects.

In [21]:
raw_df['Date'] = pd.to_datetime(raw_df['Date'])

In [40]:
one_hot = LabelBinarizer()
binary_labels = one_hot.fit_transform(raw_df['Failure'].values)

In [41]:
raw_df['Failure'] = binary_labels

In [43]:
raw_df.Operator.value_counts()

Operator2    1952
Operator8     976
Operator1     976
Operator7     976
Operator5     976
Operator3     976
Operator6     976
Operator4     976
Name: Operator, dtype: int64

In [45]:
operator_dummies = pd.get_dummies(raw_df['Operator'])

In [48]:
equipment_df = pd.concat([raw_df.drop('Operator', axis=1), operator_dummies], axis=1)